In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT_numbers_replaced_with_at_sign/dev.csv
/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT_numbers_replaced_with_at_sign/test.txt
/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT_numbers_replaced_with_at_sign/dev.txt
/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT_numbers_replaced_with_at_sign/train.csv
/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT_numbers_replaced_with_at_sign/test.csv
/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT_numbers_replaced_with_at_sign/train/train.txt
/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT/dev.csv
/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT/test.txt
/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT/dev.txt
/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT/train.csv
/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT/test.csv
/kaggle/input/pubmed-200k-rtc/PubMed_200k_RCT/train/train.txt
/kaggle/input/pubmed-200k-rtc/PubMed_20k_RCT/dev.csv
/kaggle/input/pubmed-200k-rtc/PubMed_20k_RCT/test.txt
/kaggle/input/pubmed-200k-rtc/PubMed_20k_

# Data Ingestion

In [2]:
TRAIN_FILE_LOCATION = '/kaggle/input/pubmed-200k-rtc/PubMed_20k_RCT/train.csv'
train_dataset = pd.read_csv(TRAIN_FILE_LOCATION)

print(f"Size of training set: {len(train_dataset)}")
train_dataset.head()

Size of training set: 180040


,abstract_id,line_id,abstract_text,line_number,total_lines,target
0,24293578,24293578_0_12,To investigate the efficacy of 6 weeks of dail...,0,12,OBJECTIVE
1,24293578,24293578_1_12,A total of 125 patients with primary knee OA w...,1,12,METHODS
2,24293578,24293578_2_12,Outcome measures included pain reduction and i...,2,12,METHODS
3,24293578,24293578_3_12,Pain was assessed using the visual analog pain...,3,12,METHODS
4,24293578,24293578_4_12,Secondary outcome measures included the Wester...,4,12,METHODS


In [3]:
VAL_FILE_LOCATION = '/kaggle/input/pubmed-200k-rtc/PubMed_20k_RCT/dev.csv'
val_dataset = pd.read_csv(VAL_FILE_LOCATION)

print(f"Size of validation dataset: {len(val_dataset)}")
val_dataset.head()

Size of validation dataset: 30212


,abstract_id,line_id,abstract_text,line_number,total_lines,target
0,24290286,24290286_0_10,IgE sensitization to Aspergillus fumigatus and...,0,10,BACKGROUND
1,24290286,24290286_1_10,It is not clear whether these patients would b...,1,10,BACKGROUND
2,24290286,24290286_2_10,We sought to determine whether a 3-month cours...,2,10,OBJECTIVE
3,24290286,24290286_3_10,Asthmatic patients who were IgE sensitized to ...,3,10,METHODS
4,24290286,24290286_4_10,Primary outcomes were improvement in quality o...,4,10,METHODS


In [4]:
TEST_FILE_LOCATION = '/kaggle/input/pubmed-200k-rtc/PubMed_20k_RCT/test.csv'
test_dataset = pd.read_csv(TEST_FILE_LOCATION)

print(f"Size of test dataset: {len(test_dataset)}")
test_dataset.head()

Size of test dataset: 30135


,abstract_id,line_id,abstract_text,line_number,total_lines,target
0,24845963,24845963_0_9,This study analyzed liver function abnormaliti...,0,9,BACKGROUND
1,24845963,24845963_1_9,A post hoc analysis was conducted with the use...,1,9,RESULTS
2,24845963,24845963_2_9,Liver function tests ( LFTs ) were measured at...,2,9,RESULTS
3,24845963,24845963_3_9,Survival analyses were used to assess the asso...,3,9,RESULTS
4,24845963,24845963_4_9,The percentage of patients with abnormal LFTs ...,4,9,RESULTS


# Preprocessing the datasets

## 1. Encoding the classes

In [5]:
from sklearn.preprocessing import LabelEncoder

y_encoder = LabelEncoder()
y_encoder.fit(train_dataset['target'])
CLASSES = y_encoder.classes_
CLASSES

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


array(['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVE', 'RESULTS'],
      dtype=object)

In [6]:
y_train = y_encoder.transform(train_dataset['target'])
y_train

array([3, 2, 2, ..., 4, 1, 1])

In [7]:
y_val = y_encoder.transform(val_dataset['target'])
y_val

array([0, 0, 3, ..., 4, 1, 1])

In [8]:
y_test = y_encoder.transform(test_dataset['target'])
y_test

array([0, 4, 4, ..., 4, 4, 1])

## 2. Extract the text as features

In [9]:
text_train_dataset = train_dataset['abstract_text']
text_train_dataset 

0         To investigate the efficacy of 6 weeks of dail...
1         A total of 125 patients with primary knee OA w...
2         Outcome measures included pain reduction and i...
3         Pain was assessed using the visual analog pain...
4         Secondary outcome measures included the Wester...
                                ...                        
180035    For the absolute change in percent atheroma vo...
180036    For PAV , a significantly greater percentage o...
180037    Both strategies had acceptable side effect pro...
180038    Compared with standard statin monotherapy , th...
180039    ( Plaque Regression With Cholesterol Absorptio...
Name: abstract_text, Length: 180040, dtype: object

In [10]:
from tensorflow.data import Dataset, AUTOTUNE

BATCH_SIZE = 128

def batch_dataset(X, y):
    return Dataset.from_tensor_slices((X['abstract_text'], y)) \
        .batch(BATCH_SIZE) \
        .prefetch(AUTOTUNE)

train_df = batch_dataset(train_dataset, y_train)
len(train_df), train_df

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

(1407,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [11]:
val_df = batch_dataset(val_dataset, y_val)
len(val_df), val_df

(237,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [12]:
test_df = batch_dataset(test_dataset, y_test)
len(test_df), test_df

(236,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)